# Topic modelling and word2vec analysis of /r/gaming user comments

This notebook demonstrates some text analysis of 1 year of comments on /r/gaming, using LDA topic modelling and word2vec. 

This notebook doesn't analyse the entire dataset. It analyses only the comments that include (an arbitrarily selected set of) terms relating to females, e.g., "woman", "women", "she", "girl", and so on. Thus the total comments for this data subset is 69502. 

In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import nltk

import os
from time import time
from collections import defaultdict

import numpy as np
import pandas as pd

import gensim
from gensim import corpora, utils
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers.dtmmodel import DtmModel

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib
import matplotlib.pyplot as plt

from dateutil import parser
import datetime

import random

%matplotlib inline

In [245]:
import csv

#with open('REDDIT_DATASET_ONE_YEAR_COMMENTS.csv', 'r') as f:
#  reader = csv.reader(f)
#  your_list = list(reader)

with open('REDDIT_DATASET_ONE_YEAR_COMMENTS_FEMALE_RELATED_COMMENTS.csv', 'r') as f:
  reader = csv.reader(f)
  your_list = list(reader)
    
your_list.pop(0) # remove the first element which is accidentally put there by R after exporting

['x']

In [406]:
# print(your_list[:5]) # first 5 elements

In [407]:
# len(your_list)

In [248]:
import itertools
merged_list = list(itertools.chain(*your_list))

In [957]:
# import random
# merged_list = merged_list[1:10000] # we create a small toy dataset to work with 
# merged_list = random.sample(merged_list, 100000)
# print(merged_list[:5]) # view first 5 elements

In [249]:
len(merged_list)

69502

In [250]:
comments_text = [comment for comment in merged_list]
comments_text_original = [comment for comment in merged_list]

In [251]:
# do word stemming?
from stemming.porter2 import stem

comments_text = [[stem(word) for word in sentence.split(" ")] for sentence in comments_text]
comments_text = [" ".join(sentence) for sentence in comments_text]

In [252]:
print(comments_text[:5])

["I've never read the book, but in the movi The Time Machine, he build a time machin to save his wife who die tragic year ago. But everi time he tri he fails, and he realiz it futile. If she never die he'd never have built it in the first place, so his be there to save her would be impossible.", 'Ahhh bro my grandma destroy in Bridge.  Her and all her old ladi partner use to drink and curs all night play that.  ', 'Goddamnit, I was a walk junkyard/arsen tri to amass enough crap to sell to get the 30k cap I need for the clinics... but whatev I got it and it over. I had to send piper away, she refus to work anywher and it was stick it at 98.', 'Not veri good.\n\nBut she must have boobs, so to the frontpag with you!\n\n', 'you left out make all the charact hyper sexual underag girl wear bikini armor.']


## Generate topic model

In this section we generate the topic model. There are some particular parameters that have an important role in shaping the outcome of the model. Changing these will affect the output in various ways. 

I have tested quite a few different combinations and the current output seems to give reasonable results (although we can certainly do better).

num_topics = 30 ... we specify that we want to generate 30 topics from our data
word stemming ... we use a word stemmer to reduce some words to their roots (e.g., walked and walking becomes "walk")
n-grams ... we specify that we want single terms (e.g. "walked"), and also find bi-grams or 2-grams (e.g., "she walked")
stop words ... we remove common English words that are not useful for analysis (e.g. "the", "and")
max_df ... we ignore terms that have a document frequency strictly higher than the given threshold
min_df ... ignore terms that have a document frequency strictly lower than the given threshold


In [370]:
# for good overview of these values see: http://stackoverflow.com/a/35615151/2589495
max_df = 0.03 # ignore terms that have a document frequency strictly higher than x% of docs (or occurs in > n docs if integer value)
min_df = 0.0007 # ignore terms that have a document frequency strictly less than x% of documents (or occurs in < n documents if integer value)
# Some good values found so far...
# max_df = 0.05 (smaller in same order of magnititude is also good e.g. 0.03)
# min_df = 2
# k topics = 50
# passes = 3

# bi-grams:
# tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,ngram_range=(1,2),stop_words='english',analyzer='word')
# unigrams 
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,stop_words='english')
tf = tf_vectorizer.fit_transform(comments_text)

In [371]:
# dictionary.filter_tokens(bad_ids=low_value_words)
# corpus = [dictionary.doc2bow(doc) for doc in texts]
corpus = gensim.matutils.Sparse2Corpus(tf.T)
n_terms, n_docs = corpus.sparse.shape
id2word = {i:word for i, word in enumerate(tf_vectorizer.get_feature_names())}
dictionary = Dictionary.from_corpus(corpus, id2word=id2word)

In [372]:
# generate LDA model
number_topics = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=number_topics, id2word = dictionary, passes=100,update_every=1, chunksize=10000)

In [373]:
# ldamodel.print_topics(-1)

## Visualisation of topics (by term/word)

This is an interactive visualisation of the topic model. The right-hand side shows which terms belong to each topic and how important they are for that topic (in order of importance). 

The left-hand Intertopic Distance Map gives an indication of how closely related the topics are (projected into a two-dimensional space). It may or may not be that helpful analytically, but is included anyway. 

In [374]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary,sort_topics=False)
pyLDAvis.display(vis)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [375]:
docTopicProbMat = ldamodel[corpus]

In [376]:
docTopicProbMat_list = list(docTopicProbMat)

In [377]:
# print(docTopicProbMat_list[7]) # get the topic probabilities for a document
# print(comments_text[7]) # and the document text

In [378]:
comment_probabilities = []
for doc in docTopicProbMat_list:
    docTopic = 999 # reset the topic
    bestMatch = 0 # reset the best match
    possibleMatch = 0 # reset the possible match
    for topic in doc:
        possibleMatch = topic[1]
        if possibleMatch > bestMatch:
                bestMatch = topic[1]
                # if bestMatch > 0.7: # minimum threshold for comment topic relatedness
                docTopic = topic[0]
                # print("bestMatch is now: ",bestMatch)
                # print("docTopic is now: ",docTopic)
    # print("best topic was: ",docTopic)
    comment_probabilities.append(docTopic)

In [379]:
comment_probabilities[0:10]

[7, 19, 9, 26, 26, 26, 20, 7, 9, 23]

In [380]:
comment_probabilities = [probs + 1 for probs in comment_probabilities] # now when we ask for topic n we actually get topic n (not topic n-1)

In [381]:
# print(comments_text[8])
# comment_probabilities[8]

In [382]:
comments_and_topics = pd.DataFrame(
    {'original_comment_text': comments_text_original,
     'comment_text': comments_text,
     'topic': comment_probabilities
    })

In [383]:
# comments_and_topics

In [384]:
# a function to get n random example comments assigned to a given topic k
def getTopicExamples(topic_id,num_examples):
    topic_comments = list(comments_and_topics['original_comment_text'][comments_and_topics.topic == topic_id])
    topic_comments = random.sample(topic_comments, num_examples)
    return(topic_comments)

## Extract 10 comment examples for each topic

In this section we extract 10 (randomly selected) text comments that belong to each topic. 

This provides some examples of comments that belong to each topic. The topics are ordered from 1 to k (where k is the number of topics we specified for the model). 

You can use these comment examples to help make sense of the topic terms in the above visualisation, in other words "what is going on" for each topic.

In [385]:
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,10)
    print("\n____________________ TOPIC",k,"______________________________\n")
    print(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')


____________________ TOPIC 1 ______________________________

yeah wanna know why shes an asshole? because she thinks she can sit where she wants simply because shes a girl.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Tbh if I was a girl I'd be dripping wet right now.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Lol, why does she stand like that? Is her back broken? 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
All the guys wanted to be him and all the women wanted to be with him. 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
She basically is Yennifer in every role she ever plays.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
&gt; but also find Soul Calibur's way of showing women to be borderline offensive

Lol
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Can confirm this seems to be a girl thing, my gf plays all games with invert mouse/stick.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Does she want a

In [386]:
# theta, _ = ldamodel.inference(corpus) # we can extract the thetas directly from the corpus

In [387]:
# theta[8] # columns are indexed from 0, so topic n is topic n+1 (e.g. topic22 is topic23)

## Word2vec analysis of text

In this section we do some exploratory text analysis using the novel word2vec algorithm.

This is very experimental... just some computational doodling.

In [388]:
sentences = [comment for comment in merged_list]
sentences = [[stem(word) for word in sentence.split(" ")] for sentence in sentences]

In [389]:
# import nltk
# nltk.download('stopwords')

In [390]:
from nltk.corpus import stopwords
filtered_words = [word for word in sentences if word not in stopwords.words('english')]

In [391]:
# print(filtered_words[2]) # sanity check the data first

In [392]:
# convert to lowercase
filtered_lower_words = []
for doc in filtered_words:
    filtered_lower_words.append([word.lower() for word in doc])

In [393]:
# remove punctuation
import string
filtered_lower_words_nopunct = []
for doc in filtered_lower_words:
    filtered_lower_words_nopunct.append([''.join(c for c in s if c not in string.punctuation) for s in doc])

In [394]:
# remove empty words due to previous steps 
filtered_lower_words_nopunct_nospaces = []
for doc in filtered_lower_words_nopunct:
    filtered_lower_words_nopunct_nospaces.append([s for s in doc if s])

In [395]:
print(filtered_lower_words_nopunct_nospaces[2]) # sanity check filtered_lower_words_nopunct_nospacesthe data first

['goddamnit', 'i', 'was', 'a', 'walk', 'junkyardarsen', 'tri', 'to', 'amass', 'enough', 'crap', 'to', 'sell', 'to', 'get', 'the', '30k', 'cap', 'i', 'need', 'for', 'the', 'clinics', 'but', 'whatev', 'i', 'got', 'it', 'and', 'it', 'over', 'i', 'had', 'to', 'send', 'piper', 'away', 'she', 'refus', 'to', 'work', 'anywher', 'and', 'it', 'was', 'stick', 'it', 'at', '98']


In [396]:
from gensim.models import Word2Vec
min_count = 10
size = 50
window = 4
 
model = Word2Vec(filtered_lower_words_nopunct_nospaces, min_count=min_count, size=size, window=window)


Let's try the classic word2vec example:

King - man + woman = ?

In [397]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

[('booker', 0.6980931162834167),
 ('comstock', 0.6935669183731079),
 ('queen', 0.675255537033081),
 ('wild', 0.6730973720550537),
 ('command', 0.6495621800422668),
 ('shadow', 0.6494511961936951),
 ('trench', 0.6433960199356079),
 ('fireflies', 0.6384139060974121),
 ('jedi', 0.6276146173477173),
 ('elizabeth', 0.6203716993331909)]

Using this kind of analysis we can ask other questions more specific to the paper:
    
Gamer - man + woman = ?

(What is 'gamer' without taking into account 'men', but taking into account 'women'?)

In [398]:
model.wv.most_similar(positive=['women', 'gamer'], negative=['men'])

[('nerd', 0.6684061884880066),
 ('fan', 0.643656849861145),
 ('gaming', 0.6328079104423523),
 ('hobby', 0.6300907731056213),
 ('videogam', 0.6172414422035217),
 ('hardcor', 0.6150112748146057),
 ('onlin', 0.5993642210960388),
 ('streamer', 0.5982412099838257),
 ('gamers', 0.5931899547576904),
 ('communiti', 0.5771631598472595)]

And we can also do the converse for men:

Gamer - woman + man = ?

(What is 'gamer' without taking into account women, but taking into account men?)

In [399]:
model.wv.most_similar(positive=['men', 'gamer'], negative=['women'])

[('gamers', 0.7603329420089722),
 ('streamer', 0.5821444392204285),
 ('attract', 0.5403155088424683),
 ('harass', 0.5397537350654602),
 ('protagonist', 0.5207520127296448),
 ('adult', 0.5198334455490112),
 ('gaming', 0.5129627585411072),
 ('young', 0.5076983571052551),
 ('circumcis', 0.5070184469223022),
 ('males', 0.5051846504211426)]

In [400]:
model.most_similar("angry")

[('scary', 0.7482726573944092),
 ('uncomfortable', 0.7308987379074097),
 ('hungry', 0.7245802879333496),
 ('salti', 0.724134087562561),
 ('complicated', 0.6888057589530945),
 ('naive', 0.6854866743087769),
 ('whores', 0.6832634806632996),
 ('annoying', 0.6715452075004578),
 ('pathetic', 0.6713964939117432),
 ('annoyed', 0.6707309484481812)]

In [401]:
model.most_similar(["sjw"])

[('feminist', 0.7887753248214722),
 ('idiot', 0.7881379127502441),
 ('sjws', 0.7682986855506897),
 ('bigot', 0.7282266020774841),
 ('asshol', 0.7177695035934448),
 ('bullshit', 0.7088010907173157),
 ('racist', 0.6985815763473511),
 ('tumblr', 0.6930736303329468),
 ('troll', 0.6925855875015259),
 ('sexist', 0.6892664432525635)]

We can also see which word within a set of words doesn't belong:

In [402]:
model.wv.doesnt_match("ps4 xbox pc chess".split())


'chess'

In [431]:
model.wv.doesnt_match("mouse keyboard gamepad".split())


'gamepad'

In [444]:
model.wv.doesnt_match("chick hot cosplay guy".split())

'guy'